In [1]:
# !pip install evaluate

In [2]:
from datasets import load_from_disk
# from evaluate import load
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
import gc

In [3]:
# Load pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.ctc_zero_infinity = True

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    # processor: Wav2Vec2BertProcessor
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        del input_features, label_features
        gc.collect()

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [5]:
train_set = load_from_disk("/kaggle/input/train-test-data-for-w2v2/train_data")
test_set = load_from_disk("/kaggle/input/train-test-data-for-w2v2/test_data")
train_set, test_set
# .select(range(10**2))

(Dataset({
     features: ['input_values', 'input_length', 'labels', 'path'],
     num_rows: 10000
 }),
 Dataset({
     features: ['input_values', 'input_length', 'labels', 'path'],
     num_rows: 1000
 }))

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    run_name="w2v2 10000",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    output_dir='./results',
    logging_dir='./logs',
    num_train_epochs=5,
    # bf16=True,
    fp16=True,
    learning_rate=1e-3,
    # learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    # group_by_length=False,
    # gradient_checkpointing=True,
    # save_total_limit=2,
)

In [7]:
# import numpy as np

# bleu = load("bleu")

# def compute_metrics(pred):
#     try:
#         pred_logits = pred.predictions
#         pred_ids = np.argmax(pred_logits, axis=-1)
    
#         pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    
#         pred_str = processor.batch_decode(pred_ids)
#         # we do not want to group tokens when computing the metrics
#         label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
#         bleu_val = bleu.compute(predictions=pred_str, references=label_str)
    
#         return {"bleu": bleu_val}
#     except:
#         return {"bleu": 0}

In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    # compute_metrics=compute_metrics,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=processor.feature_extractor,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [9]:
import wandb
wandb.login(key="91a0db028dce6f175361702b5140fa9c941bf8ff")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
trainer.train()

wandb: Currently logged in as: deeptanshumalu (deeptanshu-malu). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241120_041600-1uzl694m
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run w2v2 10000
wandb: ⭐️ View project at https://wandb.ai/deeptanshu-malu/huggingface
wandb: 🚀 View run at https://wandb.ai/deeptanshu-malu/huggingface/runs/1uzl694m
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was

Epoch,Training Loss,Validation Loss
0,648.674800,237.688492
1,102.126800,256.262054
2,84.653800,224.465729
4,63.478700,200.092743


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=780, training_loss=194.88935546875, metrics={'train_runtime': 12375.8624, 'train_samples_per_second': 4.04, 'train_steps_per_second': 0.063, 'total_flos': 1.0560191724171049e+19, 'train_loss': 194.88935546875, 'epoch': 4.992})

In [11]:
model.save_pretrained("./w2v2_finetuned")
processor.save_pretrained("./w2v2_finetuned")

[]